# Data Transform Dash UI
- Name: Minh T. Nguyen
- Date: 11/24/2023
- About:
    - **Description Sentiment Analysis**: Use pretrained models to performed sentimental analysis and create new feature.
    - **Features Extraction**: Based on the provided features let's create new features.
        - Use the processed sentiment dataset
    - **Clean up the dataset for classical ML modeling.**

In [1]:
!ls ../data

final_dataset_image.json     sentimental_extraction_kaggle.csv
final_dataset_no_image.json  sentimental_extraction_kaggle.json
images_sample		     sentimental_extraction_sample.csv
Kaggle-renthop.torrent	     train.json


In [2]:
!pip install --upgrade pandas

**Note:** The datasets can be found [here]((https://www.kaggle.com/competitions/two-sigma-connect-rental-listing-inquiries/data?select=train.json.zip)).
- train.json: the training set.
- images_sample.zip: listing images organized by listing_id (a sample of 100 listings)
- Kaggle-renthop.7z: listing images organized by listing_id. Total size: 78.5 GB compressed.

In [3]:
# import libraries
import numpy as np
import pandas as pd
from collections import Counter
import re
import os
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


## 1. Import dataset

In [4]:
# import the dataset (this dataset has already remove outlier)
df = pd.read_json("../data/sentimental_extraction_kaggle.json")
df.head(5)

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level,clean_description,truncated_description,sentiment,sentiment_label
4,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/7170325_3bb5ac84...,2400,145 Borinquen Place,medium,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,spacious 1 bedroom 1 bathroom in williamsburg!...,"{'label': 'POSITIVE', 'score': 0.8850623965000...",1
6,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,955db33477af4f40004820b4aed804a0,[https://photos.renthop.com/2/7092344_7663c19a...,3800,230 East 44th,low,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,brand new gut renovated true 2 bedroomfind you...,"{'label': 'POSITIVE', 'score': 0.9983743429}",1
9,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,c8b10a317b766204f08e613cef4ce7a0,[https://photos.renthop.com/2/7158677_c897a134...,3495,405 East 56th Street,medium,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,* * flex 2 bedroom with full pressurized wall ...,"{'label': 'POSITIVE', 'score': 0.9986716509}",1
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,medium,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,a brand new 3 bedroom 1. 5 bath apartmentenjoy...,"{'label': 'NEGATIVE', 'score': 0.6298918724}",0
15,1.0,0,bfb9405149bfff42a92980b594c28234,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,East 34th Street,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7439,7225292,-73.9743,2c3b41f588fbb5234d8a1e885a436cfa,[https://photos.renthop.com/2/7225292_901f1984...,2795,340 East 34th Street,low,Over-sized Studio w abundant closets. Availabl...,over - sized studio w abundant closets. availa...,"{'label': 'NEGATIVE', 'score': 0.9978052974}",0


In [5]:
print(f"There are {len(df)} samples.")

There are 48871 samples.


In [6]:
# # use the first 5 row for test only
# df = df.head(5) 

## 2. Feature Extraction

In [7]:
# check the first 3 row's features list
print(df.features.iloc[0])
print(df.features.iloc[1])
print(df.features.iloc[2])

['Dining Room', 'Pre-War', 'Laundry in Building', 'Dishwasher', 'Hardwood Floors', 'Dogs Allowed', 'Cats Allowed']
['Doorman', 'Elevator', 'Laundry in Building', 'Dishwasher', 'Hardwood Floors', 'No Fee']
['Doorman', 'Elevator', 'Laundry in Building', 'Laundry in Unit', 'Dishwasher', 'Hardwood Floors']


In [8]:
# flatten the list of features from all rows
all_features = [feature for sublist in df['features'] for feature in sublist]

# count the frequency of each feature
feature_counts = Counter(all_features)

# filter features that have a frequency above the threshold
frequency_threshold = 10000
high_freq_features = [feature for feature, count in feature_counts.items() if count >= frequency_threshold]

# create binary columns for each high-frequency feature
for feature in high_freq_features:
    df['feature_' + feature.lower()] = df['features'].apply(lambda x: 1 if feature in x else 0)

In [9]:
print(f"The features with most (>= 10k) counts are {high_freq_features}")

The features with most (>= 10k) counts are ['Laundry in Building', 'Dishwasher', 'Hardwood Floors', 'Dogs Allowed', 'Cats Allowed', 'Doorman', 'Elevator', 'No Fee', 'Fitness Center']


In [10]:
# view the cleaned dataset with extracted features
df.head(5)

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,...,sentiment_label,feature_laundry in building,feature_dishwasher,feature_hardwood floors,feature_dogs allowed,feature_cats allowed,feature_doorman,feature_elevator,feature_no fee,feature_fitness center
4,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,...,1,1,1,1,1,1,0,0,0,0
6,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,...,1,1,1,1,0,0,1,1,1,0
9,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,...,1,1,1,1,0,0,1,1,0,0
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,...,0,0,0,0,0,0,0,0,0,0
15,1.0,0,bfb9405149bfff42a92980b594c28234,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,East 34th Street,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7439,7225292,-73.9743,...,0,1,0,0,0,0,1,1,0,1


## 3. JSON to SQLite Database

In [11]:
# extract numerical features
final_df = df[['bathrooms', 'bedrooms', 'description','display_address', 'street_address', 'latitude', 'longitude', 'listing_id','price', 'sentiment_label', 'feature_laundry in building', 
               'feature_dishwasher', 'feature_hardwood floors', 'feature_dogs allowed',
               'feature_cats allowed', 'feature_doorman', 'feature_elevator',
               'feature_no fee', 'feature_fitness center', 'interest_level']]

In [12]:
final_df = final_df.rename(columns={'feature_laundry in building': 'feature_laundry_in_building'})
final_df = final_df.rename(columns={'feature_hardwood floors': 'feature_hardwood_floors'})
final_df = final_df.rename(columns={'feature_dogs allowed': 'feature_dogs_allowed'})
final_df = final_df.rename(columns={'feature_cats allowed': 'feature_cats_allowed'})
final_df = final_df.rename(columns={'feature_no fee': 'feature_no_fee'})
final_df = final_df.rename(columns={'feature_fitness center': 'feature_fitness_center'})

In [13]:
# check the dataset
final_df.head(5)

,bathrooms,bedrooms,description,display_address,street_address,latitude,longitude,listing_id,price,sentiment_label,feature_laundry_in_building,feature_dishwasher,feature_hardwood_floors,feature_dogs_allowed,feature_cats_allowed,feature_doorman,feature_elevator,feature_no_fee,feature_fitness_center,interest_level
4,1.0,1,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,145 Borinquen Place,40.7108,-73.9539,7170325,2400,1,1,1,1,1,1,0,0,0,0,medium
6,1.0,2,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,230 East 44th,40.7513,-73.9722,7092344,3800,1,1,1,1,0,0,1,1,1,0,low
9,1.0,2,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,405 East 56th Street,40.7575,-73.9625,7158677,3495,1,1,1,1,0,0,1,1,0,0,medium
10,1.5,3,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,792 Metropolitan Avenue,40.7145,-73.9425,7211212,3000,0,0,0,0,0,0,0,0,0,0,medium
15,1.0,0,Over-sized Studio w abundant closets. Availabl...,East 34th Street,340 East 34th Street,40.7439,-73.9743,7225292,2795,0,1,0,0,0,0,1,1,0,1,low


In [14]:
# replace empty strings with None
final_df = final_df.replace('', None)

In [15]:
# drop any rows with any None value
final_df = final_df.dropna()

In [16]:
# create SQLite engine
engine = create_engine('sqlite:///database.db')

# write DataFrame to SQLite table
final_df.to_sql('properties', con=engine, if_exists='replace', index=False)

47305